In [22]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import json
import os

# Job object for ease of use
class Job():
    def __init__(self, title, location):
        self.title = title
        self.location = location

def GetJobData(start: int):
    # The linkedin URL that we want to search through
    url = f"https://www.linkedin.com/jobs/search/?geoId=100902529&keywords=data&location=Flemish%20Region%2C%20Belgium&start={start}"

    # Options
    chrome_options = Options()
    chrome_options.add_argument("--headless") # So the window doesn't pop up

    # Service
    service = Service('./chromedriver')

    # using chrome cause it's popular
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    html = driver.page_source

    # Beautiful soup, such beauty
    soup = BeautifulSoup(html, "html.parser")
    # Look the HTML up on the Chrome browser since it's completely different in Opera
    jobs = soup.find_all('div', {'class', 'base-search-card__info'})
    # location class: job-card-container__metadata-item

    retval = []
    # Get the text we want and write it to a file
    for job in jobs:
        title = job.find('h3', {'class', 'base-search-card__title'}).text.strip()
        metadata = job.find('div', {'class', 'base-search-card__metadata'})
        location = metadata.find('span', {'class', 'job-search-card__location'}).text.strip()
        retval.append(Job(title, location))

    # need to close the driver
    driver.close()

    # return list
    return retval

# Remove file if it already exists
if os.path.exists('jobs.json'):
    os.remove('jobs.json')

# Main thing to do
jobs = []
for start in range(0, 1000, 25):
    list_of_jobs = GetJobData(start)
    jobs += list_of_jobs

txt_json = json.dumps([ob.__dict__ for ob in jobs])
with open('jobs.json', 'a') as f:
    f.write(txt_json)

C:\Users\Yuré\AppData\Local\Temp\ipykernel_21860\1339802221.py:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=service, chrome_options=chrome_options)


In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

url = "https://www.linkedin.com/jobs/search/?distance=25.0&geoId=100902529&keywords=data"

options = Options()
options.add_argument("--headless")

service = Service('./chromedriver')

driver = webdriver.Chrome(service=service, options=options)
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')
jobs = soup.find_all('div', {'class', 'base-search-card__info'})

for job in jobs:
    title = job.find('h3', {'class', 'base-search-card__title'}).text.strip()
    metadata = job.find('div', {'class', 'base-search-card__metadata'})
    location = metadata.find('span', {'class', 'job-search-card__location'}).text.strip()
    print(title, location)

driver.close()
